In [14]:
%matplotlib inline
from matplotlib import style
style.use('fivethirtyeight')
import matplotlib.pyplot as plt

In [15]:
import numpy as np
import pandas as pd

In [16]:
import datetime as dt

# Reflect Tables into SQLAlchemy ORM

In [17]:
# Python SQL toolkit and Object Relational Mapper
import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine, func

In [18]:
# Path to sqlite
database_path = "Resources/hawaii.sqlite"

In [19]:
# Create an engine that can talk to the database
engine = create_engine(f"sqlite:///{database_path}")

In [20]:
# Declare a Base using `automap_base()`
Base = automap_base()

# reflect an existing database into a new model and reflect the tables
Base.prepare(engine, reflect=True)

In [21]:
# Print all of the classes that automap found
Base.classes.keys()

['measurement', 'station']

In [22]:
# Save references to each table
Measurement = Base.classes.measurement
Station = Base.classes.station

In [23]:
# Create a session (link) from Python to the DB
session = Session(engine)

In [24]:
# For Measurements table - Display the row's and columns and data in dictionary format
first_row_Measurement = session.query(Measurement).first()
first_row_Measurement.__dict__

{'_sa_instance_state': <sqlalchemy.orm.state.InstanceState at 0x1d37f2eccf8>,
 'date': '2010-01-01',
 'id': 1,
 'prcp': 0.08,
 'station': 'USC00519397',
 'tobs': 65.0}

NOTE: tobs is the temperature at Time of Observation Bias (TOBS). Weather stations don’t take a single temperature measurement at the observation time. Rather, they use what are called minimum-maximum thermometers that record both maximum and minimum temperatures between resets of the instrument. The time at which the instrument is reset and the measurements are written down in the observers logbook is referred to as the time of observation.

In [25]:
# For Station table - Display the row's and columns and data in dictionary format
first_row_Station = session.query(Station).first()
first_row_Station.__dict__

{'_sa_instance_state': <sqlalchemy.orm.state.InstanceState at 0x1d3052af3c8>,
 'longitude': -157.8168,
 'name': 'WAIKIKI 717.2, HI US',
 'id': 1,
 'elevation': 3.0,
 'latitude': 21.2716,
 'station': 'USC00519397'}

# Exploratory Climate Analysis

In [29]:
# Design a query to retrieve the last 12 months of precipitation data and plot the results
# Data is from the Measurement Table where Precipitation is a float 'prcp' and Date is 'date' string in the YYYY-MM-DD format


('2010-01-01')

In [30]:
# Perform some inital analysis of dates in the Measurement table
# What is the earliest date?
session.query(Measurement.date).order_by(Measurement.date).first()

('2010-01-01')

In [32]:
# Perform some inital analysis of dates in the Measurement table
# What is the last date?
session.query(Measurement.date).order_by(Measurement.date.desc()).first()

('2017-08-23')

In [39]:
# The last 12 months would be all dates greater than '2016-08-23'
query_date = dt.date(2017,8,23) -dt.timedelta(days=365)
print("Query Date: ", query_date)


Query Date:  2016-08-23


In [46]:
# Design a query to retrieve the last 12 months of precipitation data
session.query(Measurement.date, Measurement.prcp).\
    filter(Measurement.date > query_date).all().\
    filter(Measurement.prcp None).all()

SyntaxError: invalid syntax (<ipython-input-46-6da782f0272a>, line 4)

![precipitation](Resources/Images/precipitation.png)

In [ ]:
# Calculate the date 1 year ago from the last data point in the database


In [ ]:
# Perform a query to retrieve the data and precipitation scores

In [ ]:
# Save the query results as a Pandas DataFrame and set the index to the date column

In [ ]:
# Sort the dataframe by date

In [ ]:
# Use Pandas to calcualte the summary statistics for the precipitation data

![describe](Resources/Images/describe.png)

In [ ]:
# Use Pandas Plotting with Matplotlib to plot the data


In [ ]:
# Design a query to show how many stations are available in this dataset?


In [ ]:
# What are the most active stations? (i.e. what stations have the most rows)?
# List the stations and the counts in descending order.


In [ ]:
# Using the station id from the previous query, calculate the lowest temperature recorded, 
# highest temperature recorded, and average temperature most active station?


In [ ]:
# Choose the station with the highest number of temperature observations.
# Query the last 12 months of temperature observation data for this station and plot the results as a histogram


![precipitation](Resources/Images/station-histogram.png)

In [ ]:
# This function called `calc_temps` will accept start date and end date in the format '%Y-%m-%d' 
# and return the minimum, average, and maximum temperatures for that range of dates
def calc_temps(start_date, end_date):
    """TMIN, TAVG, and TMAX for a list of dates.
    
    Args:
        start_date (string): A date string in the format %Y-%m-%d
        end_date (string): A date string in the format %Y-%m-%d
        
    Returns:
        TMIN, TAVE, and TMAX
    """
    
    return session.query(func.min(Measurement.tobs), func.avg(Measurement.tobs), func.max(Measurement.tobs)).\
        filter(Measurement.date >= start_date).filter(Measurement.date <= end_date).all()

# function usage example
print(calc_temps('2012-02-28', '2012-03-05'))

In [ ]:
# Use your previous function `calc_temps` to calculate the tmin, tavg, and tmax 
# for your trip using the previous year's data for those same dates.


In [ ]:
# Plot the results from your previous query as a bar chart. 
# Use "Trip Avg Temp" as your Title
# Use the average temperature for the y value
# Use the peak-to-peak (tmax-tmin) value as the y error bar (yerr)


In [ ]:
# Calculate the total amount of rainfall per weather station for your trip dates using the previous year's matching dates.
# Sort this in descending order by precipitation amount and list the station, name, latitude, longitude, and elevation



## Optional Challenge Assignment

In [ ]:
# Create a query that will calculate the daily normals 
# (i.e. the averages for tmin, tmax, and tavg for all historic data matching a specific month and day)

def daily_normals(date):
    """Daily Normals.
    
    Args:
        date (str): A date string in the format '%m-%d'
        
    Returns:
        A list of tuples containing the daily normals, tmin, tavg, and tmax
    
    """
    
    sel = [func.min(Measurement.tobs), func.avg(Measurement.tobs), func.max(Measurement.tobs)]
    return session.query(*sel).filter(func.strftime("%m-%d", Measurement.date) == date).all()
    
daily_normals("01-01")

In [ ]:
# calculate the daily normals for your trip
# push each tuple of calculations into a list called `normals`

# Set the start and end date of the trip

# Use the start and end date to create a range of dates

# Stip off the year and save a list of %m-%d strings

# Loop through the list of %m-%d strings and calculate the normals for each date


In [ ]:
# Load the previous query results into a Pandas DataFrame and add the `trip_dates` range as the `date` index


In [ ]:
# Plot the daily normals as an area plot with `stacked=False`
